#### création des classes

In [1]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship


Base = declarative_base()


class Client(Base):
    __tablename__ = 'clients'
    # clé primaire
    id = Column(Integer, primary_key=True)  
    nom = Column(String)
    # relation 1-N
    commandes = relationship("Commande", back_populates="client")  


class Commande(Base):
    __tablename__ = 'commandes'
    id = Column(Integer, primary_key=True)
    montant = Column(Integer)
    # clé étrangère
    client_id = Column(Integer, ForeignKey('clients.id'))  
    client = relationship("Client", back_populates="commandes")

#### Initialisation de la base de données

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# 1. Connexion à une base de données SQLite :
# L'argument 'sqlite:///ma_base_clients.db' crée un fichier 'ma_base_clients.db' 
# dans le répertoire courant s'il n'existe pas.
engine = create_engine("sqlite:///ma_base_clients.db", echo=True)

# 2. Création des tables dans la base de données
# Cette ligne regarde toutes les classes qui héritent de Base (Client, Commande) 
# et crée les tables SQL correspondantes (si elles n'existent pas déjà).
Base.metadata.create_all(engine)

# 3. Configuration de la session :
# Lie la session à notre moteur (engine) pour pouvoir interagir avec la BDD.
Session = sessionmaker(bind=engine)
session = Session()

2025-11-27 14:20:07,112 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:20:07,113 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clients")
2025-11-27 14:20:07,113 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:20:07,114 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("clients")
2025-11-27 14:20:07,114 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:20:07,115 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("commandes")
2025-11-27 14:20:07,115 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:20:07,116 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("commandes")
2025-11-27 14:20:07,116 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:20:07,117 INFO sqlalchemy.engine.Engine 
CREATE TABLE clients (
	id INTEGER NOT NULL, 
	nom VARCHAR, 
	PRIMARY KEY (id)
)


2025-11-27 14:20:07,118 INFO sqlalchemy.engine.Engine [no key 0.00057s] ()
2025-11-27 14:20:07,124 INFO sqlalchemy.engine.Engine 
CREATE TABLE commandes (
	id INTEGER 